In [1]:
!pip install transformers torch faiss-cpu scikit-learn rank_bm25 rank bm25 tqdm pyserini==0.22.1 sentence_transformers

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.6.1-py2.py3-none-any.whl.metadata (8.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.5/140.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 39.1 MB/s eta 0:00:00
Using cached pybind11-2.6.1-py2.py3-none-any.whl (188 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.2 MB/s eta 0:00:00
  Created wheel for rank: filename=rank-1.0.0-py3-none-any.whl size=1486 sha256=dc141bdaa0e2e9853830a2c67c3bf58adad2f99dfa4cd5b419380ac4b4c35dd9
  Stored in directory: /root/.cache/pip/wheels/f4/7a

In [2]:
import os
import re
import nltk
import json
import torch
import requests
import subprocess
import numpy as np
import networkx as nx
nltk.download('punkt')
nltk.download('stopwords')
from typing import Dict, List
from rank_bm25 import BM25Okapi
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from collections import defaultdict
from nltk.tokenize import word_tokenize
from transformers import BertTokenizer, BertModel
from sentence_transformers import SentenceTransformer
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import RagTokenizer, RagRetriever, RagTokenForGeneration, BertTokenizer, BertModel

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
!git clone https://github.com/RegNLP/ObliQADataset.git

Cloning into 'ObliQADataset'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 68 (delta 11), reused 47 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (68/68), 11.83 MiB | 14.04 MiB/s, done.
Resolving deltas: 100% (11/11), done.


### Retrieves Individual Docs with RANK

In [4]:
import os
import json
from rank_bm25 import BM25Okapi
from typing import List, Dict
from nltk.tokenize import word_tokenize
from collections import defaultdict

documents = []
document_ids = []
doc_id_to_text = {}
doc_id_to_tokens = defaultdict(list)

def preprocess_text(text):
    return text.lower()

# Load and process documents
for i in range(1, 41):
    with open(os.path.join("ObliQADataset/StructuredRegulatoryDocuments", f"{i}.json")) as f:
        doc = json.load(f)

        for passage in doc:
            doc_id = str(passage["DocumentID"])
            passage_text = preprocess_text(passage["Passage"])

            if doc_id not in doc_id_to_text:
                doc_id_to_text[doc_id] = passage_text
            else:
                doc_id_to_text[doc_id] += " " + passage_text

            doc_id_to_tokens[doc_id].extend(word_tokenize(passage_text))

all_documents = list(doc_id_to_tokens.values())
all_document_ids = list(doc_id_to_tokens.keys())

bm25 = BM25Okapi(all_documents, k1=2, b=1)

def retrieve_passages(user_query: str, topk: int = 10) -> List[Dict]:
    preprocessed_query = preprocess_text(user_query)
    tokenized_query = word_tokenize(preprocessed_query)

    scores = bm25.get_scores(tokenized_query)

    top_results = sorted(
        [(all_document_ids[idx], score) for idx, score in enumerate(scores)],
        key=lambda x: x[1],
        reverse=True
    )[:topk]

    results = [{"doc_id": doc_id, "rank": rank + 1, "score": score, "text": doc_id_to_text[doc_id]}
               for rank, (doc_id, score) in enumerate(top_results)]

    return results

results = retrieve_passages("Can the ADGM provide examples of legal risks associated with securitisation that Authorised Persons should particularly be aware of and manage?")
for result in results:
    print(f"Rank: {result['rank']}, Doc ID: {result['doc_id']}, Score: {result['score']}")
    print(f"Text: {result['text'][:500]}...")

Rank: 1, Doc ID: 13, Score: 36.63802651825453
Text: application, interpretation and categorisation application  subject to (2), these rules apply to every authorised person where its financial services permission authorises it to carry on one or more of the regulated activities listed in 1.3.1(a), 1.3.2(a), 1.3.3(1)(a), 1.3.4(a), 1.3.5(a), 1.3.6(a) or 1.3.7(a). in respect of a fund manager that:
(a) 	manages only venture capital funds; or
(b)	(i)	manages only venture capital funds; and
(ii)	undertakes one or both of the regulated activities of ad...
Rank: 2, Doc ID: 34, Score: 35.29035112957475
Text: scope.
this guidance is issued under section 15(2) of the financial services and markets regulations 2015 (“fsmr”).  it should be read in conjunction with fsmr, the relevant rulebooks of the financial services regulatory authority (“fsra”) and the fsra’s guidance & policies manual.
 scope.
this guidance sets out the fsra’s approach to the regulation of the trading, clearing, settlement and

### Answer Generation

In [5]:
!pip install transformers torch

## With T5 summarization

In [6]:
import os
import json
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
from collections import defaultdict
from transformers import pipeline

# Initialize QA model pipeline for generating answers
qa_pipeline = pipeline("summarization", model="t5-small")

documents = []
document_ids = []
doc_id_to_passages = defaultdict(list)
max_token_length = 512  # Max tokens per chunk for BM25

def preprocess_text(text):
    return text.lower()

# Load documents and store passages
for i in range(1, 41):
    with open(os.path.join("ObliQADataset/StructuredRegulatoryDocuments", f"{i}.json")) as f:
        doc = json.load(f)
        for passage in doc:
            doc_id = str(passage["DocumentID"])
            passage_text = preprocess_text(passage["Passage"])
            doc_id_to_passages[doc_id].append(passage_text)

# Tokenize and split into chunks for BM25
all_passages = []
passage_to_doc_map = []

for doc_id, passages in doc_id_to_passages.items():
    for passage in passages:
        tokens = word_tokenize(passage)
        for i in range(0, len(tokens), max_token_length):
            chunk = tokens[i:i + max_token_length]
            all_passages.append(chunk)
            passage_to_doc_map.append((doc_id, passage))

bm25 = BM25Okapi(all_passages, k1=2, b=1)

def retrieve_passages(user_query: str, topk: int = 5):
    preprocessed_query = preprocess_text(user_query)
    tokenized_query = word_tokenize(preprocessed_query)
    scores = bm25.get_scores(tokenized_query)

    top_results = sorted(
        zip(scores, passage_to_doc_map),
        key=lambda x: x[0],
        reverse=True
    )[:topk]

    results = [
        {
            "rank": rank + 1,
            "doc_id": doc_id,
            "score": score,
            "text": passage[:500]  # Top part of the passage for context
        }
        for rank, (score, (doc_id, passage)) in enumerate(top_results)
    ]

    return results

def generate_answer(user_query: str, retrieved_passages):
    context = " ".join([result['text'] for result in retrieved_passages])
    answer = qa_pipeline(context, truncation=True, max_length=150)[0]["summary_text"]
    return answer

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [7]:
query = "What kind of documentation and verification does the FSRA require from a Mining Reporting Entity to prove adherence to the appropriate Mining Reporting Standard when disclosing Exploration Targets and Production Targets?"

retrieved_passages = retrieve_passages(query, topk=5)
answer = generate_answer(query, retrieved_passages)

for result in retrieved_passages:
    print(f"Rank: {result['rank']}, Doc ID: {result['doc_id']}, Score: {result['score']}")
    print(f"Context: {result['text']}...")
    print()

print("Generated Answer:", answer)

Rank: 1, Doc ID: 30, Score: 80.10729864038618
Context: concepts relating to the disclosure of mining activities
mining projects and materiality
the fsra considers that the requirements for disclosing exploration results, estimates of mineral resources, estimates of ore reserves, non-equivalent estimates and production targets  only apply to information that is material.
...

Rank: 2, Doc ID: 30, Score: 79.65828398643868
Context: concepts relating to the disclosure of mining activities
what is a ‘mining reporting entity’?
there are two types of mining reporting entities under the rules, being either:

a)	a mining exploration reporting entity; or

b)	a mining production reporting entity.
...

Rank: 3, Doc ID: 30, Score: 78.77544235022484
Context: disclosures to be prepared in accordance with the mining reporting standards . the fsra considers that rules 11.2.1 and 11.2.2 are the most important rules in relation to the requirement for minerals activity disclosures within adgm.  rule 11.2.

## With FLAN T5 generator

In [9]:
class EnhancedQASystem:
    def __init__(self, model_name="google/flan-t5-large", max_token_length=512):
        self.max_token_length = max_token_length
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
        self.doc_id_to_passages = defaultdict(list)
        self.all_passages = []
        self.passage_to_doc_map = []
        self.bm25 = None

    def preprocess_text(self, text: str) -> str:
        """Enhanced text preprocessing"""
        text = re.sub(r'[^\w\s.,!?-]', '', text)  # Remove special characters
        return ' '.join(text.lower().split())  # Standardize whitespace

    def load_documents(self, data_dir: str):
        """Load and preprocess documents"""
        for i in range(1, 41):
            try:
                with open(os.path.join(data_dir, f"{i}.json")) as f:
                    doc = json.load(f)
                    for passage in doc:
                        doc_id = str(passage["DocumentID"])
                        passage_text = self.preprocess_text(passage["Passage"])
                        self.doc_id_to_passages[doc_id].append(passage_text)
            except Exception as e:
                print(f"Error loading document {i}: {str(e)}")

        self._prepare_bm25_index()

    def _prepare_bm25_index(self):
        """Prepare BM25 index with chunking"""
        for doc_id, passages in self.doc_id_to_passages.items():
            for passage in passages:
                tokens = word_tokenize(passage)
                for i in range(0, len(tokens), self.max_token_length):
                    chunk = tokens[i:i + self.max_token_length]
                    self.all_passages.append(chunk)
                    self.passage_to_doc_map.append((doc_id, ' '.join(chunk)))

        self.bm25 = BM25Okapi(self.all_passages, k1=1.5, b=0.75)

    def retrieve_passages(self, query: str, topk: int = 5) -> List[Dict]:
        """Retrieve passages based on the query"""
        preprocessed_query = self.preprocess_text(query)
        tokenized_query = word_tokenize(preprocessed_query)
        scores = self.bm25.get_scores(tokenized_query)

        top_results = sorted(
            zip(scores, self.passage_to_doc_map),
            key=lambda x: x[0],
            reverse=True
        )[:topk]

        return [
            {
                "rank": rank + 1,
                "doc_id": doc_id,
                "score": score,
                "text": passage
            }
            for rank, (score, (doc_id, passage)) in enumerate(top_results)
        ]

    def generate_answer(self, query: str, retrieved_passages: List[Dict]) -> str:
        """Generate answer from retrieved passages"""
        context = " ".join([result['text'] for result in retrieved_passages])
        prompt = (
            "Based on the following context, provide a detailed and structured answer to the question.\n\n"
            f"Question: {query}\n"
            f"Context: {context}\n"
            "Answer:"
        )
        inputs = self.tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True)

        with torch.no_grad():
            outputs = self.model.generate(
                inputs.input_ids,
                max_length=300,
                num_beams=5,
                length_penalty=1.5,
                early_stopping=True,
                no_repeat_ngram_size=3
            )

        answer = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return self._post_process_answer(answer)

    def _post_process_answer(self, answer: str) -> str:
        """Post-process the generated answer"""
        answer = re.sub(r'Question:|Context:', '', answer).strip()
        sentences = sent_tokenize(answer)
        processed_sentences = [sentence.capitalize() + '.' for sentence in sentences if sentence]
        return ' '.join(processed_sentences)

    def answer_query(self, query: str, topk: int = 5) -> Dict:
        """Comprehensive method to process query and return detailed results"""
        retrieved_passages = self.retrieve_passages(query, topk)
        answer = self.generate_answer(query, retrieved_passages)

        return {
            'answer': answer,
            'source_passages': retrieved_passages,
            'source_documents': list(set(p['doc_id'] for p in retrieved_passages))
        }

In [10]:
def main():
    qa_system = EnhancedQASystem()
    qa_system.load_documents("ObliQADataset/StructuredRegulatoryDocuments")

    query = "What kind of documentation and verification does the FSRA require from a Mining Reporting Entity to prove adherence to the appropriate Mining Reporting Standard when disclosing Exploration Targets and Production Targets?"
    results = qa_system.answer_query(query)

    print("\nGenerated Answer:", results['answer'])
    print("\nRetrieved Passages:")
    for result in results['source_passages']:
        print(f"Rank: {result['rank']}, Doc ID: {result['doc_id']}, Score: {result['score']:.4f}")
        print(f"Context: {result['text'][:500]}...")

if __name__ == "__main__":
    main()


Generated Answer: Rule 11.2.1 operates such that any disclosure relating to exploration targets, exploration results, mineral resources or ore reserves is to be prepared in accordance with a mining reporting standard.

Retrieved Passages:
Rank: 1, Doc ID: 30, Score: 75.2240
Context: disclosures to be prepared in accordance with the mining reporting standards . the fsra considers that rules 11.2.1 and 11.2.2 are the most important rules in relation to the requirement for minerals activity disclosures within adgm . rule 11.2.1 requires that any disclosure by a mining reporting entity that includes a statement about exploration targets , exploration results , mineral resources , ore reserves or production targets , must be prepared in accordance with a mining reporting standar...
Rank: 2, Doc ID: 30, Score: 74.0355
Context: production targets what can they be disclosed in relation to ? rule 11.8.34 provides that , among other things , the inferred mineral resources and exploration target

In [11]:
import os
import json
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize, sent_tokenize
from collections import defaultdict
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from typing import List, Dict
import re

class EnhancedQASystem:
    def __init__(self, model_name="google/flan-t5-large", max_token_length=512):
        self.max_token_length = max_token_length
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
        self.doc_id_to_passages = defaultdict(list)
        self.all_passages = []
        self.passage_to_doc_map = []
        self.bm25 = None

    def preprocess_text(self, text: str) -> str:
        """Enhanced text preprocessing"""
        # Remove special characters but keep important punctuation
        text = re.sub(r'[^\w\s.,!?-]', '', text)
        # Standardize whitespace
        text = ' '.join(text.split())
        return text.lower()

    def load_documents(self, data_dir: str):
        """Load and preprocess documents with better error handling"""
        for i in range(1, 41):
            try:
                with open(os.path.join(data_dir, f"{i}.json")) as f:
                    doc = json.load(f)
                    for passage in doc:
                        doc_id = str(passage["DocumentID"])
                        passage_text = self.preprocess_text(passage["Passage"])
                        self.doc_id_to_passages[doc_id].append(passage_text)
            except Exception as e:
                print(f"Error loading document {i}: {str(e)}")

        self._prepare_bm25_index()

    def _prepare_bm25_index(self):
        """Prepare BM25 index with smart chunking"""
        for doc_id, passages in self.doc_id_to_passages.items():
            for passage in passages:
                # Split into sentences for more natural chunks
                sentences = sent_tokenize(passage)
                current_chunk = []
                current_length = 0

                for sentence in sentences:
                    tokens = word_tokenize(sentence)
                    if current_length + len(tokens) > self.max_token_length:
                        if current_chunk:
                            self.all_passages.append(current_chunk)
                            self.passage_to_doc_map.append((doc_id, ' '.join(word_tokenize(' '.join(current_chunk)))))
                        current_chunk = tokens
                        current_length = len(tokens)
                    else:
                        current_chunk.extend(tokens)
                        current_length += len(tokens)

                if current_chunk:
                    self.all_passages.append(current_chunk)
                    self.passage_to_doc_map.append((doc_id, ' '.join(word_tokenize(' '.join(current_chunk)))))

        self.bm25 = BM25Okapi(self.all_passages, k1=1.5, b=0.75)

    def retrieve_passages(self, query: str, topk: int = 5) -> List[Dict]:
        """Enhanced passage retrieval with relevance scoring"""
        preprocessed_query = self.preprocess_text(query)
        tokenized_query = word_tokenize(preprocessed_query)

        # Get BM25 scores
        scores = self.bm25.get_scores(tokenized_query)

        # Enhanced scoring with query term overlap
        query_terms = set(tokenized_query)

        def calculate_relevance_score(passage_text: str, bm25_score: float) -> float:
            passage_terms = set(word_tokenize(passage_text.lower()))
            term_overlap = len(query_terms.intersection(passage_terms))
            overlap_score = term_overlap / len(query_terms)
            return bm25_score * 0.7 + overlap_score * 0.3

        # Combine scores with passage data
        scored_passages = [
            (calculate_relevance_score(passage, score), doc_id, passage)
            for score, (doc_id, passage) in zip(scores, self.passage_to_doc_map)
        ]

        # Sort and get top results
        top_results = sorted(scored_passages, key=lambda x: x[0], reverse=True)[:topk]

        return [
            {
                "rank": rank + 1,
                "doc_id": doc_id,
                "score": score,
                "text": passage
            }
            for rank, (score, doc_id, passage) in enumerate(top_results)
        ]

    def generate_answer(self, query: str, retrieved_passages: List[Dict]) -> str:
        """Enhanced answer generation with better context integration and structured output"""
        # Sort passages by score and combine context
        sorted_passages = sorted(retrieved_passages, key=lambda x: x['score'], reverse=True)
        context = " ".join([result['text'] for result in sorted_passages])

        # Create an enhanced prompt template with specific instructions
        prompt = (
            "Based on the following context, provide a detailed and structured answer to the question. "
            "Focus on specific requirements and procedures mentioned in the regulatory documents. "
            "If there are multiple requirements, list them clearly.\n\n"
            f"Question: {query}\n"
            f"Context: {context}\n"
            "Answer: Please provide a comprehensive response that directly addresses the question "
            "and includes specific details from the regulatory documents."
        )

        # Generate answer with improved parameters for regulatory content
        inputs = self.tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True)

        with torch.no_grad():
            outputs = self.model.generate(
                inputs.input_ids,
                max_length=300,
                min_length=100,
                num_beams=5,
                length_penalty=1.5,
                early_stopping=True,
                no_repeat_ngram_size=3,
                do_sample=True,
                temperature=0.6,
                top_p=0.95,
                repetition_penalty=2.5
            )

        answer = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Enhanced post-processing
        answer = self._post_process_answer(answer)
        return answer

    def _post_process_answer(self, answer: str) -> str:
        """Enhanced answer post-processing with better structure and formatting"""
        # Remove any remaining prompt artifacts
        answer = re.sub(r'Question:|Context:|Answer:', '', answer)

        # Clean up spacing and special characters
        answer = re.sub(r'\s+', ' ', answer).strip()
        answer = re.sub(r'\.+', '.', answer)
        answer = re.sub(r'\s+\.', '.', answer)

        # Improve sentence structure
        sentences = sent_tokenize(answer)
        processed_sentences = []

        for sentence in sentences:
            # Capitalize first letter of each sentence
            sentence = sentence.capitalize()
            # Remove redundant periods
            sentence = re.sub(r'\.+$', '.', sentence)
            # Ensure sentence ends with proper punctuation
            if not sentence.endswith(('.', '?', '!')):
                sentence += '.'
            processed_sentences.append(sentence)

        # Join sentences with proper spacing
        answer = ' '.join(processed_sentences)

        # Format lists if present (identified by numbers or bullet points)
        if re.search(r'\d+\.|\-|\•', answer):
            lines = answer.split('. ')
            formatted_lines = []
            for line in lines:
                if re.match(r'^\d+\.|\-|\•', line.strip()):
                    formatted_lines.append('\n' + line)
                else:
                    formatted_lines.append(line)
            answer = '. '.join(formatted_lines)

        return answer

    def answer_query(self, query: str, topk: int = 5) -> Dict:
        """Comprehensive method to process query and return detailed results"""
        retrieved_passages = self.retrieve_passages(query, topk)
        answer = self.generate_answer(query, retrieved_passages)

        return {
            'answer': answer,
            'source_passages': retrieved_passages,
            'source_documents': list(set(p['doc_id'] for p in retrieved_passages))
        }

def main():
    qa_system = EnhancedQASystem()
    qa_system.load_documents("ObliQADataset/StructuredRegulatoryDocuments")

    query = "What kind of documentation and verification does the FSRA require from a Mining Reporting Entity to prove adherence to the appropriate Mining Reporting Standard when disclosing Exploration Targets and Production Targets?"

    retrieved_passages = qa_system.retrieve_passages(query, topk=5)
    answer = qa_system.generate_answer(query, retrieved_passages)

    print("\nRetrieved Passages:")
    for result in retrieved_passages:
        print(f"\nRank: {result['rank']}, Doc ID: {result['doc_id']}, Score: {result['score']:.4f}")
        print(f"Context: {result['text'][:200]}...")

    print("\nGenerated Answer:", answer)

if __name__ == "__main__":
    main()


Retrieved Passages:

Rank: 1, Doc ID: 30, Score: 52.8222
Context: disclosures to be prepared in accordance with the mining reporting standards . the fsra considers that rules 11.2.1 and 11.2.2 are the most important rules in relation to the requirement for minerals ...

Rank: 2, Doc ID: 30, Score: 51.9977
Context: production targets what can they be disclosed in relation to ? rule 11.8.34 provides that , among other things , the inferred mineral resources and exploration target as applicable do not feature as a...

Rank: 3, Doc ID: 30, Score: 51.3196
Context: non-equivalent estimates when can non-equivalent estimates be disclosed ? rule 11.2.1 operates such that any disclosure relating to exploration targets , exploration results , mineral resources or ore...

Rank: 4, Doc ID: 30, Score: 50.3803
Context: concepts relating to the disclosure of mining activities what is a mining reporting entity ? there are two types of mining reporting entities under the rules , being either a a mining